In [ ]:
# Sources
# https://www.statsmodels.org/stable/mixed_linear.html

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statistics
from scipy import stats

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Junior/Dyanne JP/ABCD_Release4.0_Tabular_Dataset.csv')
df_baseline = df[df['eventname'] == 'baseline_year_1_arm_1']
df_2year = df[df['eventname'] == '2_year_follow_up_y_arm_1']

In [ ]:
baseline_2year = pd.merge(df_baseline, df_2year, on='subjectkey', how='left', suffixes=('_baseline', '_2year'))

In [ ]:
baseline_2year.dropna(subset=['eventname_2year'], inplace=True)

In [ ]:
check = ['nihtbx_picvocab_uncorrected_baseline','nihtbx_picvocab_uncorrected_2year','nihtbx_flanker_uncorrected_baseline',
         'nihtbx_flanker_uncorrected_2year','nihtbx_pattern_uncorrected_baseline','nihtbx_pattern_uncorrected_2year',
         'nihtbx_picture_uncorrected_baseline','nihtbx_picture_uncorrected_2year','nihtbx_reading_uncorrected_baseline',
         'nihtbx_reading_uncorrected_2year','nihtbx_cryst_uncorrected_baseline','nihtbx_cryst_uncorrected_2year']

cleaned = baseline_2year.dropna(subset=check)
#(7172,1055)

# drop columns that only have NaN values
cleaned.dropna(axis=1, how='all', inplace=True)
#(7172,892)

print('Number of participants with all 5 test scores')
print(cleaned.shape[0])

In [ ]:
non_numeric_columns = cleaned.select_dtypes(exclude=['number']).columns

print("Non-numeric columns:", non_numeric_columns.tolist())

In [ ]:
(cleaned['sex_baseline']==cleaned['sex_2year']).sum()

In [ ]:
cleaned = cleaned.drop('sex_2year',axis=1)

In [ ]:
cleaned['sex_baseline'] = cleaned['sex_baseline'].replace({'M': 0, 'F': 1})

In [ ]:
from sklearn.impute import SimpleImputer

def medianimpute(df):
    # Create a copy of the DataFrame to avoid changing the original data
    df_imputed = df.copy()

    # Identify numeric columns by data type
    numeric_cols = df_imputed.select_dtypes(include=[np.number]).columns

    # Define the imputer with a median strategy
    imputer = SimpleImputer(strategy='median')

    # Apply the imputer only to the numeric columns
    df_imputed[numeric_cols] = imputer.fit_transform(df_imputed[numeric_cols])

    return df_imputed

In [ ]:
imputed = medianimpute(cleaned)

In [ ]:
# nest family in ABCD study site for LMM random effect
imputed['site_family'] = imputed['abcd_site_baseline'].astype(str) + "_" + imputed['family_id_baseline'].astype(str)
imputed['site_family'] = imputed['abcd_site_2year'].astype(str) + "_" + imputed['family_id_baseline'].astype(str)

In [ ]:
df_low = imputed[(imputed['income_baseline'] >= 1) & (imputed['income_baseline'] < 7)]
df_med = imputed[imputed['income_baseline'].isin([7])]
df_high = imputed[(imputed['income_baseline'] > 7) & (imputed['income_baseline'] <= 10)]

In [ ]:
low_diff = pd.DataFrame({
    'low_diff_picvocab': df_low['nihtbx_picvocab_uncorrected_2year'] - df_low['nihtbx_picvocab_uncorrected_baseline'],
    'low_diff_flanker': df_low['nihtbx_flanker_uncorrected_2year'] - df_low['nihtbx_flanker_uncorrected_baseline'],
    'low_diff_pattern': df_low['nihtbx_pattern_uncorrected_2year'] - df_low['nihtbx_pattern_uncorrected_baseline'],
    'low_diff_picture': df_low['nihtbx_picture_uncorrected_2year'] - df_low['nihtbx_picture_uncorrected_baseline'],
    'low_diff_reading': df_low['nihtbx_reading_uncorrected_2year'] - df_low['nihtbx_reading_uncorrected_baseline']
})

high_diff = pd.DataFrame({
    'high_diff_picvocab': df_high['nihtbx_picvocab_uncorrected_2year'] - df_high['nihtbx_picvocab_uncorrected_baseline'],
    'high_diff_flanker': df_high['nihtbx_flanker_uncorrected_2year'] - df_high['nihtbx_flanker_uncorrected_baseline'],
    'high_diff_pattern': df_high['nihtbx_pattern_uncorrected_2year'] - df_high['nihtbx_pattern_uncorrected_baseline'],
    'high_diff_picture': df_high['nihtbx_picture_uncorrected_2year'] - df_high['nihtbx_picture_uncorrected_baseline'],
    'high_diff_reading': df_high['nihtbx_reading_uncorrected_2year'] - df_high['nihtbx_reading_uncorrected_baseline']
})

df_low = pd.concat([df_low, low_diff], axis=1)
df_high = pd.concat([df_high, high_diff], axis=1)

In [ ]:
low_picvocab_neg = df_low[df_low['low_diff_picvocab']<0]
high_picvocab_neg = df_high[df_high['high_diff_picvocab']<0]
low_picvocab_pos = df_low[df_low['low_diff_picvocab']>0]
high_picvocab_pos = df_high[df_high['high_diff_picvocab']>0]

low_flanker_neg = df_low[df_low['low_diff_flanker']<0]
high_flanker_neg = df_high[df_high['high_diff_flanker']<0]
low_flanker_pos = df_low[df_low['low_diff_flanker']>0]
high_flanker_pos = df_high[df_high['high_diff_flanker']>0]

low_pattern_neg = df_low[df_low['low_diff_pattern']<0]
high_pattern_neg = df_high[df_high['high_diff_pattern']<0]
low_pattern_pos = df_low[df_low['low_diff_pattern']>0]
high_pattern_pos = df_high[df_high['high_diff_pattern']>0]

low_picture_neg = df_low[df_low['low_diff_picture']<0]
high_picture_neg = df_high[df_high['high_diff_picture']<0]
low_picture_pos = df_low[df_low['low_diff_picture']>0]
high_picture_pos = df_high[df_high['high_diff_picture']>0]

low_reading_neg = df_low[df_low['low_diff_reading']<0]
high_reading_neg = df_high[df_high['high_diff_reading']<0]
low_reading_pos = df_low[df_low['low_diff_reading']>0]
high_reading_pos = df_high[df_high['high_diff_reading']>0]

In [ ]:
def drop_non_numeric(df):
    return df.select_dtypes(include=[np.number])

In [ ]:
def standardize(df):
    # Select numeric columns only.
    numeric_columns = df.select_dtypes(include=[np.number])
    # Subtract the mean and divide by the std dev for each column.
    standardized_df = (numeric_columns - numeric_columns.mean()) / numeric_columns.std()
    # If you want to keep non-numeric columns unchanged, combine back the non-numeric columns:
    non_numeric_columns = df.select_dtypes(exclude=[np.number])
    return pd.concat([standardized_df, non_numeric_columns], axis=1)

In [ ]:
low_picvocab_neg = standardize(low_picvocab_neg)
high_picvocab_neg = standardize(high_picvocab_neg)
low_picvocab_pos = standardize(low_picvocab_pos)
high_picvocab_pos = standardize(high_picvocab_pos)

low_flanker_neg = standardize(low_flanker_neg)
high_flanker_neg = standardize(high_flanker_neg)
low_flanker_pos = standardize(low_flanker_pos)
high_flanker_pos = standardize(high_flanker_pos)

low_pattern_neg = standardize(low_pattern_neg)
high_pattern_neg = standardize(high_pattern_neg)
low_pattern_pos = standardize(low_pattern_pos)
high_pattern_pos = standardize(high_pattern_pos)

low_picture_neg = standardize(low_picture_neg)
high_picture_neg = standardize(high_picture_neg)
low_picture_pos = standardize(low_picture_pos)
high_picture_pos = standardize(high_picture_pos)

low_reading_neg = standardize(low_reading_neg)
high_reading_neg = standardize(high_reading_neg)
low_reading_pos = standardize(low_reading_pos)
high_reading_pos = standardize(high_reading_pos)

In [ ]:
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", message="Random effects covariance is singular")
warnings.filterwarnings("ignore", message="The MLE may be on the boundary of the parameter space")

In [ ]:
model = smf.mixedlm("nihtbx_picvocab_uncorrected_2year ~ upps_premeditation_baseline + screentime_wkend_sns_baseline + upps_negative_baseline + euro_baseline + good_school_baseline + cbcl_internal_2year + vol_2year + upps_perseverence_baseline + worryauto_baseline + screentime_wkday_y_baseline + height_2year + upps_positive_baseline + parent_age_baseline + screentime_wkday_p_baseline + prenatal_alcohol_eff_before_baseline + scz_metaauto_baseline + screentime_maturegames_baseline + bpm_total_t_2year + screentime_addict_p_2year + total_ratio_baseline + gay_youth_2year + cpeur2_baseline + eaeur1_baseline + BIS_2year + religion_prefer_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=low_picvocab_neg, groups=low_picvocab_neg["abcd_site_2year"])
result = model.fit()
print(result.summary())

In [ ]:
model = smf.mixedlm("nihtbx_picvocab_uncorrected_2year ~ upps_premeditation_baseline + screentime_wkend_sns_baseline + upps_negative_baseline + euro_baseline + good_school_baseline + cbcl_internal_2year + vol_2year + upps_perseverence_baseline + worryauto_baseline + screentime_wkday_y_baseline + height_2year + upps_positive_baseline + parent_age_baseline + screentime_wkday_p_baseline + prenatal_alcohol_eff_before_baseline + scz_metaauto_baseline + screentime_maturegames_baseline + bpm_total_t_2year + screentime_addict_p_2year + total_ratio_baseline + gay_youth_2year + cpeur2_baseline + eaeur1_baseline + BIS_2year + religion_prefer_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=high_picvocab_neg, groups=high_picvocab_neg["abcd_site_2year"])
result = model.fit()
print(result.summary())

In [ ]:
model = smf.mixedlm("nihtbx_picvocab_uncorrected_2year ~ upps_premeditation_baseline + screentime_wkend_sns_baseline + upps_negative_baseline + euro_baseline + good_school_baseline + cbcl_internal_2year + vol_2year + upps_perseverence_baseline + worryauto_baseline + screentime_wkday_y_baseline + height_2year + upps_positive_baseline + parent_age_baseline + screentime_wkday_p_baseline + prenatal_alcohol_eff_before_baseline + scz_metaauto_baseline + screentime_maturegames_baseline + bpm_total_t_2year + screentime_addict_p_2year + total_ratio_baseline + gay_youth_2year + cpeur2_baseline + eaeur1_baseline + BIS_2year + religion_prefer_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=low_picvocab_pos, groups=low_picvocab_pos["abcd_site_2year"])
result = model.fit()
print(result.summary())

In [ ]:
model = smf.mixedlm("nihtbx_picvocab_uncorrected_2year ~ upps_premeditation_baseline + screentime_wkend_sns_baseline + upps_negative_baseline + euro_baseline + good_school_baseline + cbcl_internal_2year + vol_2year + upps_perseverence_baseline + worryauto_baseline + screentime_wkday_y_baseline + height_2year + upps_positive_baseline + parent_age_baseline + screentime_wkday_p_baseline + prenatal_alcohol_eff_before_baseline + scz_metaauto_baseline + screentime_maturegames_baseline + bpm_total_t_2year + screentime_addict_p_2year + total_ratio_baseline + gay_youth_2year + cpeur2_baseline + eaeur1_baseline + BIS_2year + religion_prefer_baseline + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=high_picvocab_pos, groups=high_picvocab_pos["abcd_site_2year"])
result = model.fit()
print(result.summary())

In [ ]:
model = smf.mixedlm("nihtbx_reading_uncorrected_2year ~ rh_adi_wsum1_baseline + good_school_2year + risk4pcauto_baseline + cbcl_totprob_baseline + BAS_Drive_2year + aspauto_baseline + anxietyauto_baseline + cbcl_external_baseline + BAS_Fun_2year + ghappimeaneur1_baseline + Cfriends_boys_2year + screentime_wkday_p_baseline + cbcl_external_2year + screentime_risk_p_2year + upps_positive_2year + vol_2year + screentime_maturegames_baseline + BIS_baseline + upps_negative_2year + height_2year + ghappihealth6_baseline + risk_electro_nicotine_y_2year + prosocial_p_baseline + parent_age_baseline + screentime_wkday_y_baseline + BAS_RR_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=low_reading_neg, groups=low_reading_neg["abcd_site_2year"])
result = model.fit()
print(result.summary())

In [ ]:
model = smf.mixedlm("nihtbx_reading_uncorrected_2year ~ rh_adi_wsum1_baseline + good_school_2year + risk4pcauto_baseline + cbcl_totprob_baseline + BAS_Drive_2year + aspauto_baseline + anxietyauto_baseline + cbcl_external_baseline + BAS_Fun_2year + ghappimeaneur1_baseline + Cfriends_boys_2year + screentime_wkday_p_baseline + cbcl_external_2year + screentime_risk_p_2year + upps_positive_2year + vol_2year + screentime_maturegames_baseline + BIS_baseline + upps_negative_2year + height_2year + ghappihealth6_baseline + risk_electro_nicotine_y_2year + prosocial_p_baseline + parent_age_baseline + screentime_wkday_y_baseline + BAS_RR_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=high_reading_neg, groups=high_reading_neg["abcd_site_2year"])
result = model.fit()
print(result.summary())

In [ ]:
model = smf.mixedlm("nihtbx_reading_uncorrected_2year ~ rh_adi_wsum1_baseline + good_school_2year + risk4pcauto_baseline + cbcl_totprob_baseline + BAS_Drive_2year + aspauto_baseline + anxietyauto_baseline + cbcl_external_baseline + BAS_Fun_2year + ghappimeaneur1_baseline + Cfriends_boys_2year + screentime_wkday_p_baseline + cbcl_external_2year + screentime_risk_p_2year + upps_positive_2year + vol_2year + screentime_maturegames_baseline + BIS_baseline + upps_negative_2year + height_2year + ghappihealth6_baseline + risk_electro_nicotine_y_2year + prosocial_p_baseline + parent_age_baseline + screentime_wkday_y_baseline + BAS_RR_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=low_reading_pos, groups=low_reading_pos["abcd_site_2year"])
result = model.fit()
print(result.summary())

In [ ]:
model = smf.mixedlm("nihtbx_reading_uncorrected_2year ~ rh_adi_wsum1_baseline + good_school_2year + risk4pcauto_baseline + cbcl_totprob_baseline + BAS_Drive_2year + aspauto_baseline + anxietyauto_baseline + cbcl_external_baseline + BAS_Fun_2year + ghappimeaneur1_baseline + Cfriends_boys_2year + screentime_wkday_p_baseline + cbcl_external_2year + screentime_risk_p_2year + upps_positive_2year + vol_2year + screentime_maturegames_baseline + BIS_baseline + upps_negative_2year + height_2year + ghappihealth6_baseline + risk_electro_nicotine_y_2year + prosocial_p_baseline + parent_age_baseline + screentime_wkday_y_baseline + BAS_RR_2year + income_baseline + high_educ_baseline + sex_baseline + race_ethnicity_baseline + age_2year + married_baseline + bmi_2year", data=high_reading_pos, groups=high_reading_pos["abcd_site_2year"])
result = model.fit()
print(result.summary())